# SimNIBS for tDCS

In [1]:
import simnibs
import numpy as np
import json

In [ ]:
# Initalize a session
s = simnibs.sim_struct.SESSION()

# Configure session

# Name of head mesh
s.fnamehead = '...simnibs_ernie\simnibs_examples\ernie\ernie.msh'
# Output folder
s.pathfem = ''
# set output 
s.fields = 'veEjJ'
# Turn off gmsh viewer
s.open_in_gmsh = False

# map the fields to the middle gray matter surface (needed for ROI analysis using atlas)
s.map_to_surf = True

In [4]:
# Initialize a tDCS simulation
tdcslist = s.add_tdcslist()
# Set currents (A)
tdcslist.currents = [1e-3, 1e-3, -1e-3, -1e-3] # 1mA

# Initialize the anode1
anode1 = tdcslist.add_electrode()
# Connect electrode to first channel
anode1.channelnr = 1
# Electrode dimension (mm)
anode1.dimensions = [8, 8]
# circular shape
anode1.shape = 'ellipse'
# 5mm thickness of eeg gel and 2 mm of eletrode
anode1.thickness = [5, 2]
# Electrode Position
anode1.centre = 'F3'

# Initialize the anode2
anode2 = tdcslist.add_electrode()
anode2.channelnr = 2
anode2.dimensions = [8, 8]
anode2.shape = 'ellipse'
anode2.thickness = [5, 2]
anode2.centre = 'F4'

# Initialize the cathode1
cathode1 = tdcslist.add_electrode()
cathode1.channelnr = 3
cathode1.dimensions = [8, 8]
cathode1.shape = 'ellipse'
cathode1.thickness = [5, 2]
cathode1.centre = [-69.99, -44.69, -48.49]

# Initialize the cathode2
cathode2 = tdcslist.add_electrode()
cathode2.channelnr = 4
cathode2.dimensions = [8, 8]
cathode2.shape = 'ellipse'
cathode2.thickness = [5, 2]
cathode2.centre = [61.64, -54.66, -44.16]

In [7]:
%%time
# run simulation
simnibs.run_simnibs(s)

### ROI Analysis

#### Using brain atlas
calculate the mean electric field in a gray matter ROI defined using an atlas

In [ ]:
# Read simulation result file
# use the middle cortical surfaces
gm_surf = simnibs.read_msh('')

# Load atlas --> DK40
atlas = simnibs.subject_atlas('DK40', '...simnibs_ernie\simnibs_examples\ernie\m2m_ernie')

EF_result = {}
for areas, values in atlas.items():
    # define ROI 
    if 'unknown' in areas or '???' in areas:
        continue
    roi = atlas[areas]
    
    # calculate nodes areas in ROI for averaging
    node_areas = gm_surf.nodes_areas()
    if not node_areas[roi].any():
        print(areas, node_areas[roi])
        EF_result[areas] = 0
        continue

    # Calculate mean field
    # check all available field with gm_surf.field
    field_name = 'E_normal'
    mean_normE = np.average(gm_surf.field[field_name][roi], weights=node_areas[roi])
    EF_result[areas] = mean_normE

### Write result into file

In [12]:
result = '../result.json'
with open(result, 'w') as f: 
    json.dump(EF_result, f)